# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [35]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [36]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [37]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [38]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [39]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

# Answer to Question 1

In [40]:
import pandas as pd
import numpy as np

# Put names of all countries and their infant_mortality stats in a dictionary
mortality_dict = {}
for element in document.iter('country'):
    country = element.find('name').text
    for subelement in element.iter('infant_mortality'):
        if pd.isnull(subelement):
            mortality_dict[country] = np.nan
        elif subelement.text == '0':
            mortality_dict[country] = 0
        else:
            mortality_dict[country] = float(subelement.text)

# Turn the dictionary into a dataframe
mortality_df = pd.DataFrame(list(mortality_dict.items()), 
                            columns=['Country','Infant_Mortality'])

# Sort and output the 10 countries with the lowest Infant_Mortality rate
mortality_df.sort_values(by='Infant_Mortality').head(10)

,Country,Infant_Mortality
50,Monaco,1.81
122,Japan,2.13
215,Norway,2.48
37,Bermuda,2.48
205,Singapore,2.53
217,Sweden,2.60
153,Czech Republic,2.63
44,Hong Kong,2.73
88,Macao,3.13
126,Iceland,3.15


# Answer to Question 2

In [41]:
# Put names of all countries and their population stats in a list
population_list = []
group = []
for country in root.findall('country'):
    try:
        group = [country.find('name').text, int(country.find('population').text)]
        population_list.append(group)
    except:
        pass

# Turn the list into a dataframe
columns = ['Country', 'Population']
pop_df = pd.DataFrame(columns=columns)
pop_df = pop_df.append(pd.DataFrame(population_list, columns=columns), ignore_index=True)

# Sort and output the 10 countries with the largest population
pop_df.sort_values(by='Population', ascending=False).head(10)

,Country,Population
55,China,543776080.0
67,India,238396327.0
120,United States,157813040.0
23,Russia,102798657.0
98,Japan,82199470.0
88,Indonesia,72592192.0
11,Germany,68230796.0
176,Brazil,53974725.0
53,United Kingdom,50616012.0
7,France,40502513.0


# Begin Question 3 Work

In [42]:
# Parse XML and place countries, ethnic groups, and percentages into lists
replace = 'na'
entries = []
for element in document.iter('country'):
    country = element.find('name').text
    for subelement in element.iter('ethnicgroup'):
        group = []
        group.append(country)
        
        if  pd.isnull(subelement):    
            continue
        else:
            group.append(subelement.text)
            if pd.isnull(subelement.attrib['percentage']): 
                group.append(replace)
            else:
                group.append(subelement.attrib['percentage'])

        entries.append(group)

# Transform the list above into a DataFrame and join with pop_df        
ethnic_df = pd.DataFrame(columns=['Country', 'Ethnic_Group', 'Percent_Pop'])
ethnic_df = ethnic_df.append(pd.DataFrame(entries, columns=['Country', 'Ethnic_Group', 'Percent_Pop']), 
                             ignore_index=True)
ethnic_df = ethnic_df.merge(pop_df, how='left', on='Country')
ethnic_df['Group_Pop_Num'] = (ethnic_df['Percent_Pop'].astype(float)/100) * ethnic_df['Population']

ethnic_df

,Country,Ethnic_Group,Percent_Pop,Population,Group_Pop_Num
0,Albania,Albanian,95,1214489.0,1.153765e+06
1,Albania,Greek,3,1214489.0,3.643467e+04
2,Greece,Greek,93,1096810.0,1.020033e+06
3,Macedonia,Macedonian,64.2,808724.0,5.192008e+05
4,Macedonia,Albanian,25.2,808724.0,2.037984e+05
5,Macedonia,Turkish,3.9,808724.0,3.154024e+04
6,Macedonia,Gypsy,2.7,808724.0,2.183555e+04
7,Macedonia,Serb,1.8,808724.0,1.455703e+04
8,Serbia,Serb,82.9,6732256.0,5.581040e+06
9,Serbia,Montenegrin,0.9,6732256.0,6.059030e+04


# Answer to Question 3

In [43]:
# Group ethnic_df by ethnic_group to obtain total population counts
ethnic_df_grouped = ethnic_df.groupby('Ethnic_Group').sum()
ethnic_df_grouped.sort_values(by='Group_Pop_Num', ascending=False, inplace=True)

# Display 10 ethnic groups with the largest overall populations
ethnic_df_grouped.head(10)

,Population,Group_Pop_Num
Ethnic_Group,,
Han Chinese,543776080.0,4.975551e+08
European,362717873.0,1.928658e+08
Indo-Aryan,238396327.0,1.716454e+08
Russian,202263854.0,9.275844e+07
African,357529690.0,8.632937e+07
Japanese,82199470.0,8.170627e+07
German,145710759.0,6.623219e+07
Dravidian,238396327.0,5.959908e+07
English,50616012.0,4.231499e+07


## Build Question 4 Resources

In [44]:
# Create and display a dataframe linking country codes to country names
country_codes = {}
for country in root.findall('country'):
    country_codes[country.attrib['car_code']] = country.find('name').text

columns=['Country_Code', 'Country']
country_codes_df = pd.DataFrame(data=list(country_codes.items()), columns=columns)
country_codes_df

,Country_Code,Country
0,ETH,Ethiopia
1,THA,Thailand
2,MK,Macedonia
3,TL,Timor-Leste
4,KN,Saint Kitts and Nevis
5,ANG,Angola
6,VN,Vietnam
7,RG,Guinea
8,RCH,Chile
9,NCA,New Caledonia


# Begin Question 4(a) Work

In [45]:
# Begin to parse XML on the river node

river_list = []
group = []
for river in root.findall('river'):
    try:
        group = [river.attrib['country'], river.find('name').text, int(river.find('length').text)]
        river_list.append(group)
    except:
        pass

# Create a dataframe containing country code, river name, and river length    
columns = ['Country_Code', 'River_Name', 'River_Length']
river_df = pd.DataFrame(columns=columns)
river_df = river_df.append(pd.DataFrame(river_list, columns=columns), ignore_index=True)

# Display resulting dataframe
river_df

,Country_Code,River_Name,River_Length
0,IS,Thjorsa,230.0
1,IS,Joekulsa a Fjoellum,206.0
2,N,Glomma,604.0
3,N,Lagen,322.0
4,S,Goetaaelv,93.0
5,N S,Klaraelv,460.0
6,S,Umeaelv,470.0
7,S,Dalaelv,520.0
8,S,Vaesterdalaelv,320.0
9,S,Oesterdalaelv,241.0


In [46]:
# As we can see above, some rivers are associated with multiple country codes
# Create a new, expanded dataframe so that each river has one country code association

expanded_river_list = []
for row in river_df.itertuples(): 
    if len(row[1].split(' ')) == 1:
        entry = [row[1], row[2], row[3]]
        expanded_river_list.append(entry)
    else:
        for code in row[1].split(' '):
            entry = [code, row[2], row[3]]
            expanded_river_list.append(entry)

expanded_river_df = pd.DataFrame(columns=['Country_Code', 'River_Name', 'River_Length'])
expanded_river_df = expanded_river_df.append(pd.DataFrame(expanded_river_list, columns=['Country_Code', 'River_Name', 'River_Length']), 
                             ignore_index=True)

# Display new, expanded dataframe
expanded_river_df

,Country_Code,River_Name,River_Length
0,IS,Thjorsa,230.0
1,IS,Joekulsa a Fjoellum,206.0
2,N,Glomma,604.0
3,N,Lagen,322.0
4,S,Goetaaelv,93.0
5,N,Klaraelv,460.0
6,S,Klaraelv,460.0
7,S,Umeaelv,470.0
8,S,Dalaelv,520.0
9,S,Vaesterdalaelv,320.0


# Answer to Question 4(a)

In [47]:
# Merge the country code and expanded river dataframes together
# Sort the dataframe by river length and country name
# Display the river with the greatest length
country_river_df = expanded_river_df.merge(country_codes_df, how='left', on='Country_Code')
print('The Amazonas river is the longest, and is associated with Brazil, Colombia, and Peru.')
country_river_df.sort_values(by=['River_Length', 'Country'], ascending=[False, True]).head(3)

The Amazonas river is the longest, and is associated with Brazil, Colombia, and Peru.


,Country_Code,River_Name,River_Length,Country
295,BR,Amazonas,6448.0,Brazil
294,CO,Amazonas,6448.0,Colombia
296,PE,Amazonas,6448.0,Peru


# Begin Question 4(b) Work

In [48]:
# Begin to parse XML on the lake node

lake_list = []
group = []
for lake in root.findall('lake'):
    try:
        group = [lake.attrib['country'], lake.find('name').text, int(lake.find('area').text)]
        lake_list.append(group)
    except:
        pass

# Create a dataframe containing country code, lake name, and lake area    
columns = ['Country_Code', 'Lake_Name', 'Lake_Area']
lake_df = pd.DataFrame(columns=columns)
lake_df = lake_df.append(pd.DataFrame(lake_list, columns=columns), ignore_index=True)

# Display resulting dataframe
lake_df

,Country_Code,Lake_Name,Lake_Area
0,SF,Inari,1040.0
1,SF,Oulujaervi,928.0
2,SF,Kallavesi,472.0
3,SF,Saimaa,4370.0
4,SF,Paeijaenne,1118.0
5,N,Mjoesa-See,368.0
6,S,Storuman,173.0
7,S,Siljan,290.0
8,S,Maelaren,1140.0
9,S,Vaenern,5648.0


In [49]:
# As we can see above, some lakes are associated with multiple country codes
# Create a new, expanded dataframe so that each lake has one country code association

expanded_lake_list = []
for row in lake_df.itertuples(): 
    if len(row[1].split(' ')) == 1:
        entry = [row[1], row[2], row[3]]
        expanded_lake_list.append(entry)
    else:
        for code in row[1].split(' '):
            entry = [code, row[2], row[3]]
            expanded_lake_list.append(entry)

expanded_lake_df = pd.DataFrame(columns=['Country_Code', 'Lake_Name', 'Lake_Area'])
expanded_lake_df = expanded_lake_df.append(pd.DataFrame(expanded_lake_list, columns=['Country_Code', 'Lake_Name', 'Lake_Area']), 
                             ignore_index=True)

# Display new, expanded dataframe
expanded_lake_df

,Country_Code,Lake_Name,Lake_Area
0,SF,Inari,1040.0
1,SF,Oulujaervi,928.0
2,SF,Kallavesi,472.0
3,SF,Saimaa,4370.0
4,SF,Paeijaenne,1118.0
5,N,Mjoesa-See,368.0
6,S,Storuman,173.0
7,S,Siljan,290.0
8,S,Maelaren,1140.0
9,S,Vaenern,5648.0


# Answer to Question 4(b)

In [50]:
# Merge the country code and expanded lake dataframes together
# Sort the dataframe by lake area and country name
# Display the lake with the greatest area
country_lake_df = expanded_lake_df.merge(country_codes_df, how='left', on='Country_Code')
print('The Caspian Sea is the largest, and is associated with Azerbaijan, Iran, Kazakhstan, Russia, and Turkmenistan.')
country_lake_df.sort_values(by=['Lake_Area', 'Country'], ascending=[False, True]).head(5)

The Caspian Sea is the largest, and is associated with Azerbaijan, Iran, Kazakhstan, Russia, and Turkmenistan.


,Country_Code,Lake_Name,Lake_Area,Country
57,AZ,Caspian Sea,386400.0,Azerbaijan
59,IR,Caspian Sea,386400.0,Iran
58,KAZ,Caspian Sea,386400.0,Kazakhstan
56,R,Caspian Sea,386400.0,Russia
60,TM,Caspian Sea,386400.0,Turkmenistan


# Begin Question 4(c) Work

In [51]:
# Begin to parse XML on the airport node

airport_list = []
group = []
for airport in root.findall('airport'):
    try:
        group = [airport.attrib['country'], airport.find('name').text, int(airport.find('elevation').text)]
        airport_list.append(group)
    except:
        pass

# Create a dataframe containing country code, airport name, and airport elevation    
columns = ['Country_Code', 'Airport_Name', 'Airport_Elevation']
airport_df = pd.DataFrame(columns=columns)
airport_df = airport_df.append(pd.DataFrame(airport_list, columns=columns), ignore_index=True)

# Display resulting dataframe
airport_df

,Country_Code,Airport_Name,Airport_Elevation
0,AFG,Herat,977.0
1,AFG,Kabul Intl,1792.0
2,AL,Tirana Rinas,38.0
3,DZ,Cheikh Larbi Tebessi,811.0
4,DZ,Batna Airport,822.0
5,DZ,Soummam,6.0
6,DZ,Tamanrasset,1377.0
7,DZ,Biskra,88.0
8,DZ,Mohamed Boudiaf Intl,691.0
9,DZ,Ain Arnat Airport,1024.0


In [52]:
# Merge the country code and airport dataframes together
# Sort the dataframe by airport elevation and country name
# Display the airport with the highest elevation
country_airport_df = airport_df.merge(country_codes_df, how='left', on='Country_Code')
print('El Alto International Airport sits at the highest elevation, and is located in Bolivia.')
country_airport_df.sort_values(by=['Airport_Elevation', 'Country'], ascending=[False, True]).head(1)

El Alto International Airport sits at the highest elevation, and is located in Bolivia.


,Country_Code,Airport_Name,Airport_Elevation,Country
80,BOL,El Alto Intl,4063.0,Bolivia
